### Problem 2:

In [1]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torchvision
import torchvision.transforms
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
import pprint

In [ ]:
class Classifier(nn.Module):
    """Convnet Classifier"""
    def __init__(self, feature_maps, dropouts):
        super(Classifier, self).__init__()
        self.feature_maps = feature_maps
        self.dropouts = dropouts
        layers = self.layer(1,feature_maps[0], dropouts[0])+self.layer(feature_maps[0],feature_maps[1], dropouts[1])+self.layer(feature_maps[1],feature_maps[2], dropouts[2])+self.layer(feature_maps[2],feature_maps[3], dropouts[3])+self.layer(feature_maps[3],feature_maps[4], dropouts[4])
        self.conv = nn.Sequential(*layers)
        self.clf = nn.Linear(feature_maps[-1], 10)
        self.drop = nn.Dropout(p=0.4)
        
        
        
    def layer(self, i, o, dropout=0.1):
        return [nn.Conv2d(in_channels=i, out_channels=o, kernel_size=(4, 4), padding=2),
                #nn.BatchNorm2d(o),
                nn.Dropout(p=dropout),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2), stride=2)]
    
    def num_parameters(self):
        pars = 0
        for a,b in zip([1] + self.feature_maps, self.feature_maps):
            current_layer_parameters = a * 16 * b
            pars = pars + current_layer_parameters
        pars = pars + self.feature_maps[-1] * 10
        print("we have ", float(pars)/1000000, " million parameters") # Ignore biases
    
    def forward(self, x):
        x = self.conv(x)
        y = x.squeeze()
        return self.clf(self.drop(y))

In [ ]:
perf = {}

torch.manual_seed(0)
mnist_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
mnist_train = torchvision.datasets.MNIST(root='./data', train=True, transform=mnist_transforms, download=True)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False, transform=mnist_transforms, download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=64, shuffle=True, num_workers=2)

cuda_available = torch.cuda.is_available()
print("Cuda available: %s" % cuda_available)
feature_maps = [[20,40,60,80,128], [20,40,60,50,40], [20,40,50,60,70]]
dropouts = [[0, 0.1, 0.1, 0.1,0.1 ], [0, 0.4, 0.4, 0.1,0.1 ], [0, 0.1, 0.5, 0.1,0.5 ]]
for feature_map in feature_maps:
    for dropout in dropouts:
        hyper = str(feature_map)+" "+str(dropout)
        print(hyper)
        perf[hyper] = []
        clf = Classifier(feature_map, dropout)
        clf.num_parameters()
        if cuda_available:
            clf = clf.cuda()
        optimizer = torch.optim.Adam(clf.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss() #  LogSoftmax and NLLLoss

        for epoch in range(10):
            # Train
            current_epoch_loss = []
            for batch_idx, (inputs, targets) in enumerate(train_loader):
                if cuda_available:
                    inputs, targets = inputs.cuda(), targets.cuda()

                optimizer.zero_grad()
                loss = criterion(clf(inputs), targets)
                loss.backward()
                optimizer.step()
                current_epoch_loss.append(loss.item())
            myLoss =  np.mean(current_epoch_loss)
            
            #    if batch_idx%500==0:
            #       print('Epoch : %d, Loss : %.3f ' % (epoch, np.mean(losses)))
            
            # Evaluate
            clf.eval()
            total = 0
            correct = 0
            for batch_idx, (inputs, targets) in enumerate(test_loader):
                if cuda_available:
                    inputs, targets = inputs.cuda(), targets.cuda()
                outputs = clf(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += targets.size(0)
                correct += predicted.eq(targets.data).cpu().sum()
            accur = 100*float(correct)/total    
            #print('Epoch : %d, Test Accuracy : %.2f%%, number of tests : %d' % (epoch, accur, total))
            #print('--------------------------------------------------------------')
            clf.train()
            perf[hyper].append((accur, myLoss))
        print(perf[hyper])


        
 


Cuda available: True
[20, 40, 60, 80, 128] [0, 0.1, 0.1, 0.1, 0.1]
we have  0.29344  million parameters
[(97.96, 0.3144947297569277), (98.86, 0.07668959631173532), (99.05, 0.05425320533332604), (99.19, 0.04697827352429313), (99.0, 0.03525888407702194), (99.1, 0.032066629496591684), (99.16, 0.027506706296905145), (99.09, 0.024175951048843007), (99.25, 0.022246938287171284), (99.16, 0.020036977282894063)]
[20, 40, 60, 80, 128] [0, 0.4, 0.4, 0.1, 0.1]
we have  0.29344  million parameters
[(98.27, 0.329681209886252), (98.76, 0.09770918964569185), (98.97, 0.0694704822171281), (99.14, 0.060814434575087735), (99.11, 0.052461012558483365), (99.05, 0.047244481386930576), (99.31, 0.04391476701794148), (99.23, 0.03966553975095246), (99.19, 0.03760211830879293), (99.32, 0.0366828586560672)]
[20, 40, 60, 80, 128] [0, 0.1, 0.5, 0.1, 0.5]
we have  0.29344  million parameters
[(98.56, 0.38310466095336526), (99.05, 0.08670556062121572), (98.89, 0.06535159540511588), (99.06, 0.05312962978204557), (99.27

# we select the  top combination of hyperparameters in the following:  
and further plot the training error and validation accuracy for the ten epochs. The training error for each epoch indicates the mean of all the losses at each step during that epoch. 

In [ ]:
best_performance = max(perf.values(), key=lambda x: max([a[0] for a in x]))
best_accuracy = [x[0] for x in best_performance]
best_loss = [x[1] for x in best_performance]


In [ ]:
def plotThis(x, label ):
    xnew = np.linspace(np.arange(x.shape[0]).min(),np.arange(x.shape[0]).max(),300)
    spl = make_interp_spline(np.arange(x.shape[0]),x, k=3)
    power_smooth = spl(xnew)
    xx = plt.plot(xnew,power_smooth, label = label)


In [ ]:
plotThis(np.array(best_accuracy), "accuracy")

plt.title("validation accuracy")
plt.xlabel("epoch")
plt.legend()
plt.show()

In [ ]:
plotThis(np.array(best_loss), "error")

plt.title("training error")
plt.legend()
plt.xlabel("epoch")
plt.show()

We have lowered the dropout from p=0.5 to p=0.1 or p=0.4 for various convolutional layers. This was inspired by Park et.al[1]. For the given problem, we find out that the above combination of dropout probabilites for different convolutional layers works the best for the purpose of reducing bias and making the model more generalizable. However, we know that presumably increasing the dropout probabilites may in 